<a href="https://colab.research.google.com/github/harix1606/human-hashing/blob/master/pswd_hashing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
import numpy as np
import csv

Using TensorFlow backend.


In [2]:
pswds = []
with open ('cue_pin.csv','r') as csvFile:
    read = csv.reader(csvFile, delimiter = ',')
    for rows in read:
        pswds.append(rows[0])

pswds = pswds[1:]
print (pswds)

['ampoesterfee', 'dskldsreedog', 'autwom6org', 'aecsicjuioyc', 'dskldsreedog', 'ffeeepeatrla', 'otseocarlddoexis', 'csebroeraecu', 'alafalcefafe', 'ficttelivsin', 'Azznflnflthe', 'nsoutiemosro', 'nvinme', 'digitalduster57', 'oaratberanch', 'oRaWhaRamina', 'Iroowbrncboo', 'sibsensponeo', 'mazo1dorm2laniro3ormebe4', 'aptl5105', 'Olarvoolarvo', 'setrsedarenc', 'epiocollorth', 'Olarvoolarvo', 'ndodogpenlca', 'mazo1dorm2laniro3ormebe4', 'Errnssryaeep', 'Dpuarkairabl', 'ikswofsairbe', 'mkidlps', 'Ifinorgenhel', 'Ratatmatmamt', 'ymuearneyrhe', 'eraerbtroeer', 'redtysdardou', 'Serpplalaign', 'nymelfindshp', 'digitaldesign58', 'oaratberanch', 'foofballerys', 'Pleeabuicarw', 'Bleblehorabl', 'elberrseseel', 'cheuittbicuit', 'ficobvtemary', 'Entylogpatsf', 'footcransery', 'footcransery', 'Gootride', 'KAUNDIPENOTT', 'Ardrseaptnnscebo', 'apprryeahmilcat', 'hirtspchaabl', 'u1eu', 'Pl@s$nky', 'feista 0235', 'getdogticeti', 'diesperfar', 'friquerietht', 'tuailauanlac5831', 'verrroosever', 'rryhorqwbott

In [3]:
#Padding the passwords:

longest_pswd = 0
for words in pswds:
    if len(words) > longest_pswd:
        longest_pswd = len(words)

padded_pswds = []
for words in pswds: 
    remaining = longest_pswd - len(words)
    new_words = '0'*remaining + words
    padded_pswds.append(new_words)

print (padded_pswds)

['000000000000ampoesterfee', '000000000000dskldsreedog', '00000000000000autwom6org', '000000000000aecsicjuioyc', '000000000000dskldsreedog', '000000000000ffeeepeatrla', '00000000otseocarlddoexis', '000000000000csebroeraecu', '000000000000alafalcefafe', '000000000000ficttelivsin', '000000000000Azznflnflthe', '000000000000nsoutiemosro', '000000000000000000nvinme', '000000000digitalduster57', '000000000000oaratberanch', '000000000000oRaWhaRamina', '000000000000Iroowbrncboo', '000000000000sibsensponeo', 'mazo1dorm2laniro3ormebe4', '0000000000000000aptl5105', '000000000000Olarvoolarvo', '000000000000setrsedarenc', '000000000000epiocollorth', '000000000000Olarvoolarvo', '000000000000ndodogpenlca', 'mazo1dorm2laniro3ormebe4', '000000000000Errnssryaeep', '000000000000Dpuarkairabl', '000000000000ikswofsairbe', '00000000000000000mkidlps', '000000000000Ifinorgenhel', '000000000000Ratatmatmamt', '000000000000ymuearneyrhe', '000000000000eraerbtroeer', '000000000000redtysdardou', '000000000000Serppl

In [4]:
#The vocabulary of the passwords
vocab = list({l for word in padded_pswds for l in word})
len_of_vocab = len(vocab)

# Mappings of character to index number and vice versa
index_to_char = {}
char_to_index = {}
for i, char in enumerate(vocab):
    index_to_char[i] = char
    char_to_index[char] = i

print (char_to_index)

{'$': 0, 'i': 1, 'W': 2, 'S': 3, 'q': 4, 'z': 5, ' ': 6, 'B': 7, '8': 8, 'o': 9, 'j': 10, 'N': 11, 'T': 12, '2': 13, 'D': 14, 'G': 15, 'R': 16, '4': 17, '1': 18, 'w': 19, 't': 20, 'h': 21, 'k': 22, 'l': 23, 'y': 24, 'a': 25, '0': 26, '7': 27, '3': 28, 'e': 29, 'I': 30, '5': 31, 'c': 32, 'n': 33, 'K': 34, 'P': 35, 'm': 36, 'v': 37, 'p': 38, 'x': 39, 'A': 40, 'U': 41, '@': 42, 'E': 43, '6': 44, 'g': 45, 'u': 46, 'O': 47, 'd': 48, 'b': 49, 's': 50, 'r': 51, 'f': 52}


In [5]:
#password sequences must be encoded as a sequence of integers.

sequences = []
for words in padded_pswds:
    encoded_seq = [char_to_index[letters] for letters in words]
    sequences.append(encoded_seq)

print (sequences)

[[26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 25, 36, 38, 9, 29, 50, 20, 29, 51, 52, 29, 29], [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 48, 50, 22, 23, 48, 50, 51, 29, 29, 48, 9, 45], [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 25, 46, 20, 19, 9, 36, 44, 9, 51, 45], [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 25, 29, 32, 50, 1, 32, 10, 46, 1, 9, 24, 32], [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 48, 50, 22, 23, 48, 50, 51, 29, 29, 48, 9, 45], [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 52, 52, 29, 29, 29, 38, 29, 25, 20, 51, 23, 25], [26, 26, 26, 26, 26, 26, 26, 26, 9, 20, 50, 29, 9, 32, 25, 51, 23, 48, 48, 9, 29, 39, 1, 50], [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 32, 50, 29, 49, 51, 9, 29, 51, 25, 29, 32, 46], [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 25, 23, 25, 52, 25, 23, 32, 29, 52, 25, 52, 29], [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 52, 1, 32, 20, 20, 29, 23, 1, 37, 50, 1, 33], [26, 26, 26, 26, 26, 26, 26, 26, 26

In [6]:
#Splitting into input and output

sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

sequences = [to_categorical(x, num_classes=len_of_vocab) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=len_of_vocab)

y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
# define model
model = Sequential()
model.add(LSTM(20, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(len_of_vocab, activation='softmax'))

print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 20)                5920      
_________________________________________________________________
dense_4 (Dense)              (None, 53)                1113      
Total params: 7,033
Trainable params: 7,033
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=20, verbose=2)

Epoch 1/20
 - 0s - loss: 3.9749 - accuracy: 0.0000e+00
Epoch 2/20
 - 0s - loss: 3.9654 - accuracy: 0.0000e+00
Epoch 3/20
 - 0s - loss: 3.9568 - accuracy: 0.0290
Epoch 4/20
 - 0s - loss: 3.9485 - accuracy: 0.0435
Epoch 5/20
 - 0s - loss: 3.9404 - accuracy: 0.0290
Epoch 6/20
 - 0s - loss: 3.9321 - accuracy: 0.0290
Epoch 7/20
 - 0s - loss: 3.9235 - accuracy: 0.0725
Epoch 8/20
 - 0s - loss: 3.9144 - accuracy: 0.0580
Epoch 9/20
 - 0s - loss: 3.9041 - accuracy: 0.0725
Epoch 10/20
 - 0s - loss: 3.8929 - accuracy: 0.1159
Epoch 11/20
 - 0s - loss: 3.8799 - accuracy: 0.1159
Epoch 12/20
 - 0s - loss: 3.8632 - accuracy: 0.1014
Epoch 13/20
 - 0s - loss: 3.8426 - accuracy: 0.1159
Epoch 14/20
 - 0s - loss: 3.8150 - accuracy: 0.1014
Epoch 15/20
 - 0s - loss: 3.7763 - accuracy: 0.1014
Epoch 16/20
 - 0s - loss: 3.7194 - accuracy: 0.0870
Epoch 17/20
 - 0s - loss: 3.6516 - accuracy: 0.0870
Epoch 18/20
 - 0s - loss: 3.5720 - accuracy: 0.0870
Epoch 19/20
 - 0s - loss: 3.5088 - accuracy: 0.0870
Epoch 20/20
 